In [ ]:
import pandas as pd
from google.cloud import bigquery
# プロジェクトIDの指定
project_id = 'inunaki'
# authが終わっていたら普通にクライアント認証は通る
client = bigquery.Client(project=project_id)

In [ ]:
query = """
WITH senge_data AS (
  -- 宣言データについて
  SELECT  DISTINCT customer_id, 1 AS senge_flg
  FROM `inunaki.dwd_action_0001.dwd_karte_user_use_coupon` 
  WHERE ym BETWEEN "2023-09-01" AND "2023-09-19"
),
coupon_data AS(
  -- クーポン一覧を見た顧客
  SELECT DISTINCT customer_id, 1 AS coupon_view_flg
  FROM `inunaki.dwd_action_0001.dwd_karte_user_view_couponlist`
  WHERE ym BETWEEN "2023-09-01" AND "2023-09-19"
),
home_data AS(
  -- ホームを見た顧客
  SELECT DISTINCT customer_id, 1 AS home_flg
  FROM `inunaki.dwd_action_0001.dwd_karte_supay_accesshome`
  WHERE ym BETWEEN "2023-09-01" AND "2023-09-19"
)
SELECT customercd, COALESCE(home_flg, 0) AS home_flg, COALESCE(coupon_view_flg, 0) AS coupon_view_flg, COALESCE(senge_flg, 0) AS senge_flg
FROM home_data
LEFT JOIN coupon_data
ON home_data.customer_id = coupon_data.customer_id
LEFT JOIN senge_data
ON home_data.customer_id = senge_data.customer_id
LEFT JOIN `inunaki.dim_crm_0001.dim_crm_customer_md5` md5_cus
ON home_data.customer_id = md5_cus.customercd_md5

"""
# #データフレームに変換
df = client.query(query).to_dataframe()

df = df.rename(columns={"customercd":"会員ID","home_flg":"ホーム利用の有無","coupon_view_flg":"クーポンタブの有無","senge_flg":"宣言の有無"})
df.to_csv("9月までのSU-PAY利用状況.csv",index=False, encoding="utf_8_sig")